In [93]:
from collections import Counter, defaultdict
from typing import *
from itertools import combinations, cycle
from parse import parse
import re

import black
import jupyter_black

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [25]:
# Day 1: Chronal Calibration
def first_repeat(frequencies):
    freqs = cycle(frequencies)
    found = set()
    current = 0
    while current not in found:
        found.add(current)
        current += next(freqs)
    return current


lines = open("2018/1.txt").read().splitlines()
frequencies = [int(line) for line in lines]
print(f"Part 1: {sum(frequencies)}")  # 411
print(f"Part 2: {first_repeat(frequencies)}")  # 56360

Part 1: 411
Part 2: 56360


In [87]:
# Day 2: Inventory Management System
def checksum(ids):
    def contains(text, repeated):
        "`text` contains exactly `repeated` repeated characters"
        return repeated in Counter(text).values()

    twos = sum(contains(id, 2) for id in ids)
    threes = sum(contains(id, 3) for id in ids)
    return twos * threes


def common_letters(ids):
    "Common letters for the two ids that only differ by one character."
    for a_, b_ in combinations(ids, 2):
        common = [a for a, b in zip(a_, b_) if a == b]
        if len(a_) - len(common) == 1:
            return "".join(common)


ids = open("2018/2.txt").read().splitlines()
print(f"Part 1: {checksum(ids)}")  # 5166
print(f"Part 2: {common_letters(ids)}")  # cypueihajytordkgzxfqplbwn

Part 1: 5166
Part 2: cypueihajytordkgzxfqplbwn


In [120]:
# Day 3: No Matter How You Slice It
def patch(row, column, width, height):
    "Return a set with all coordinates"
    return {
        (r, c) for c in range(column, column + width) for r in range(row, row + height)
    }


def point_claims(claims):
    "Number of claims per point"
    patches = Counter()
    for _, column, row, width, height in claims:
        patches.update(patch(row, column, width, height))
    return patches


def overlapping_inches(claims):
    return sum(p > 1 for p in point_claims(claims).values())


def intact_claim_id(claims):
    overlapping = {
        coord for coord, overlaps in point_claims(claims).items() if overlaps > 1
    }

    for id, column, row, width, height in claims:
        claim = patch(row, column, width, height)
        if all(point not in overlapping for point in claim):
            return id


# ID, column, row, width, height
lines = open("2018/3.txt").read().splitlines()
claims = [tuple(int(x) for x in re.findall(r"\d+", line)) for line in lines]
print(f"Part 1: {overlapping_inches(claims)}")  # 115348
print(f"Part 2: {intact_claim_id(claims)}")  # 188

Part 1: 115348
Part 2: 188


In [105]:
# Day 4: Repose Record
def sleep_schedule(lines: List[str]) -> DefaultDict[int, Counter]:
    log: List[List[str]] = sorted([line[1:].split("] ") for line in lines])
    guard = snooze = 0
    schedule = defaultdict(Counter)
    for timestamp, entry in log:
        if p := parse("Guard #{:d} begins shift", entry):
            guard = p[0]
        elif p := parse("falls asleep", entry):
            snooze = int(timestamp[-2:])
        elif p := parse("wakes up", entry):
            wakes = int(timestamp[-2:])
            assert wakes > snooze
            schedule[guard].update(range(snooze, wakes))
        else:
            raise ValueError("Can't parse", entry)
    return schedule


def part1(schedule: DefaultDict[int, Counter]) -> int:
    """
    Find the guard that has the most minutes asleep. What minute does that guard spend
    asleep the most?
    """
    guard, _ = sorted(
        [(guard, slept.total()) for guard, slept in schedule.items()],
        key=lambda x: x[1],
    )[-1]
    minute = schedule[guard].most_common(1)[0][0]

    return guard * minute


def part2(schedule: DefaultDict[int, Counter]) -> int:
    """
    Of all guards, which guard is most frequently asleep on the same minute?
    """
    guard, (minute, _) = sorted(
        [(guard, slept.most_common(1)[0]) for guard, slept in schedule.items()],
        key=lambda x: x[1][1],
    )[-1]

    return guard * minute


lines: str = open("2018/4.txt").read().splitlines()
schedule: DefaultDict[int, Counter] = sleep_schedule(lines)
print(f"Part 1: {part1(schedule)}")  # 35184
print(f"Part 2: {part2(schedule)}")  # 37886

Part 1: 35184
Part 2: 37886


In [ ]:
sorted()